In [ ]:
import xarray as xr
import numpy as np
from dask.distributed import Client, LocalCluster

cluster=  LocalCluster(n_workers=10, memory_limit='8GiB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 80,Total memory: 80.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33460,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 80
Started: Just now,Total memory: 80.00 GiB
Comm: tcp://127.0.0.1:35071,Total threads: 8
Dashboard: http://127.0.0.1:34014/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:42713,


2025-02-13 11:57:35,711 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:33598 (pid=129159) exceeded 95% memory budget. Restarting...
2025-02-13 11:57:37,849 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33598' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('array-54b527eef6845e28fa58a311e4bd4300', 86), ('array-54b527eef6845e28fa58a311e4bd4300', 31), ('array-54b527eef6845e28fa58a311e4bd4300', 5), ('array-54b527eef6845e28fa58a311e4bd4300', 40), ('array-54b527eef6845e28fa58a311e4bd4300', 59), ('array-54b527eef6845e28fa58a311e4bd4300', 68), ('array-54b527eef6845e28fa58a311e4bd4300', 77)} (stimulus_id='handle-worker-cleanup-1739444257.8491817')
2025-02-13 11:57:37,873 - distributed.nanny - WARNING - Restarting worker
2025-02-13 11:58:15,697 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:45270 (pid=131083) exceeded 95% memory budget. Restarting...
2025-02-13 11:58:17,212 - distributed.scheduler - 

In [2]:
era5 = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-512x256_equiangular_conservative.zarr',
    chunks=dict(lon=50, lat=50, time=4*120, level=1),
    storage_options=dict(token='anon'),
)

/tmp/ipykernel_129041/550085691.py:1: UserWarning: The specified chunks separate the stored chunks along dimension "level" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  era5 = xr.open_zarr(


In [18]:
precip = era5['total_precipitation_6hr']*1000
precip_out = precip.sel(latitude=slice(0,90)).resample(time='D').sum(keep_attrs=True).rename('daily_precipitation').to_dataset()
precip_out['mean_daily_precipitation'] = precip_out.daily_precipitation.weighted(np.cos(np.deg2rad(precip_out.latitude))).mean(['longitude','latitude','time'])
precip_out['std_daily_precipitation'] = precip_out.daily_precipitation.weighted(np.cos(np.deg2rad(precip_out.latitude))).std(['longitude','latitude','time'])

precip_out.assign_coords(longitude=(((precip_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/pr.nc')


In [19]:
iwv = era5['total_column_water_vapour']
iwv_out = iwv.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('total_column_water_vapour').to_dataset()
iwv_out['mean_total_column_water_vapour'] = iwv_out.total_column_water_vapour.weighted(np.cos(np.deg2rad(iwv_out.latitude))).mean(['longitude','latitude','time'])
iwv_out['std_total_column_water_vapour'] = iwv_out.total_column_water_vapour.weighted(np.cos(np.deg2rad(iwv_out.latitude))).std(['longitude','latitude','time'])

iwv_out.assign_coords(longitude=(((iwv_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/tcwv.nc')

In [29]:
uwind = era5['u_component_of_wind'].sel(level=850, drop=True).rename('u850')
uwind_out = uwind.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('u850').to_dataset()
uwind_out['mean_u850'] = uwind_out.u850.weighted(np.cos(np.deg2rad(uwind_out.latitude))).mean(['longitude','latitude','time'])
uwind_out['std_u850'] = uwind_out.u850.weighted(np.cos(np.deg2rad(uwind_out.latitude))).std(['longitude','latitude','time'])

uwind_out.assign_coords(longitude=(((uwind_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/u850.nc')

2025-02-13 12:46:12,249 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.66 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:46:14,449 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.56 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:46:15,983 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [30]:
vwind = era5['v_component_of_wind'].sel(level=850, drop=True).rename('v850')
vwind_out = vwind.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('v850').to_dataset()
vwind_out['mean_v850'] = vwind_out.v850.weighted(np.cos(np.deg2rad(vwind_out.latitude))).mean(['longitude','latitude','time'])
vwind_out['std_v850'] = vwind_out.v850.weighted(np.cos(np.deg2rad(vwind_out.latitude))).std(['longitude','latitude','time'])

vwind_out.assign_coords(longitude=(((vwind_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/v850.nc')

2025-02-13 12:55:05,337 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.63 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:55:08,092 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.63 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:55:18,144 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [22]:
z500 = era5['geopotential'].sel(level=850, drop=True).rename('z500')
z500_out = z500.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('z500').to_dataset()
z500_out['mean_z500'] = z500_out.z500.weighted(np.cos(np.deg2rad(z500_out.latitude))).mean(['longitude','latitude','time'])
z500_out['std_z500'] = z500_out.z500.weighted(np.cos(np.deg2rad(z500_out.latitude))).std(['longitude','latitude','time'])

z500_out.assign_coords(longitude=(((z500_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/z500.nc')